In [1]:
import warnings 
warnings.filterwarnings(action='ignore')

import numpy as np
import pandas as pd
import tensorflow as tf
import re
import os
import nltk
import datetime

from gensim.models.fasttext import FastText
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize

from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.utils import plot_model
from tensorflow.keras.optimizers import Adam
from tqdm import tqdm_notebook

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# 데이터 불러오기

raw_train = pd.read_csv('/content/drive/MyDrive/[데이콘] 소설 작가 분류 AI 경진대회/data/train.csv')
raw_test = pd.read_csv('/content/drive/MyDrive/[데이콘] 소설 작가 분류 AI 경진대회/data/test_x.csv')
sample_submission = pd.read_csv('/content/drive/MyDrive/[데이콘] 소설 작가 분류 AI 경진대회/data/sample_submission.csv')

In [4]:
train = raw_train.copy()
test = raw_test.copy()

In [13]:
def alpha_num(text):
    return re.sub(r'[^A-Za-z0-9 ]', '', text)


def remove_stopwords(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in stopwords:
            final_text.append(i.strip())
    return " ".join(final_text)


stopwords = ['dr', 'o', 'the', 'a']
# stopwords = [ "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", 
#              "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", 
#              "did", "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", 
#              "have", "having", "he", "he'd", "he'll", "he's", "her", "here", "here's", "hers", "herself", "him", "himself", 
#              "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "it", "it's", "its", "itself", 
#              "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought", "our", "ours", 
#              "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", "so", "some", "such", "than", "that", 
#              "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's", "these", "they", "they'd", "they'll", 
#              "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up", "very", "was", "we", "we'd", "we'll", 
#              "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's", "which", "while", "who", "who's", "whom", 
#              "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves"]

In [14]:
train['text'] = train['text'].str.lower().apply(alpha_num).apply(remove_stopwords)
test['text'] = test['text'].str.lower().apply(alpha_num).apply(remove_stopwords)

In [15]:
X_train = train['text'].values
X_test = test['text'].values
y = train['author'].values.reshape(-1, 1)
print(X_train.shape, X_test.shape, y.shape)

(54879,) (19617,) (54879, 1)


In [ ]:
count_by_author = []

for idx in range(5):
    df = train[train['author']==idx]['text'].values
    temp_tokenizer = Tokenizer()

    temp_tokenizer.fit_on_texts(df)
    temp_dict = temp_tokenizer.word_counts
    temp_dict = sorted(temp_dict.items(), key=lambda x : x[1], reverse=True)
    print('작가 {}의 유니크한 단어수 {}' .format(idx, len(temp_dict)))
    count_by_author.append(dict(temp_dict))

작가 0의 유니크한 단어수 20602
작가 1의 유니크한 단어수 15410
작가 2의 유니크한 단어수 21014
작가 3의 유니크한 단어수 19186
작가 4의 유니크한 단어수 17799


In [16]:
max_length = 500
padding_type='post'
vocab_size= 15000

In [17]:
tokenizer = Tokenizer(num_words = vocab_size,oov_token='oov')
tokenizer.fit_on_texts(X_train)
word_index = tokenizer.word_index
word_counts = tokenizer.word_counts

train_sequences = tokenizer.texts_to_sequences(X_train)
test_sequences = tokenizer.texts_to_sequences(X_test)

x_train = pad_sequences(train_sequences, padding=padding_type, maxlen=max_length)
x_test = pad_sequences(test_sequences, padding=padding_type, maxlen=max_length)
print(x_train.shape, x_test.shape)

(54879, 500) (19617, 500)


In [ ]:
sorted(word_counts.items(), key=lambda x : x[1], reverse=True)

## 1. Glove 사용하기

In [41]:
import numpy as np
embedding_dict = dict()
f = open('/content/drive/MyDrive/Glove/glove.6B.200d.txt', encoding="utf8")

for line in f:
    word_vector = line.split()
    word = word_vector[0]
    word_vector_arr = np.asarray(word_vector[1:], dtype='float32') # 100개의 값을 가지는 array로 변환
    embedding_dict[word] = word_vector_arr
f.close()
print('%s개의 Embedding vector가 있습니다.' % len(embedding_dict))

400000개의 Embedding vector가 있습니다.


In [42]:
embedding_dim = 200
embedding_matrix = np.zeros( (len(word_index)+1, embedding_dim) )

In [43]:
# 임베딩테이블 만들기
for word, idx in word_index.items():
    embedding_vector = embedding_dict.get(word)

    if embedding_vector is not None :
        embedding_matrix[idx] = embedding_vector
    else :
        embedding_matrix[idx] = np.zeros((1, embedding_dim))

In [44]:
# 임베딩 테이블
embedding_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.20327   ,  0.47347999,  0.050877  , ..., -0.21358   ,
        -0.62248999,  0.14386   ],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [45]:
target_col = 'author'
n_class = 5
seed = 42

In [46]:
import tensorflow.keras.backend as K
# Mish 활성화 함수
def mish(x):
    return x * K.tanh(K.softplus(x))

In [47]:

def get_model() :

    from tensorflow.keras import Sequential
    from tensorflow.keras.layers import Dense, Embedding, GlobalMaxPooling1D, Conv1D, Dropout, Flatten, MaxPool1D, GlobalAveragePooling1D, Flatten

    model = Sequential()
    model.add(Embedding(vocab_size, embedding_dim, weights=[embedding_matrix[0:vocab_size]], input_length=max_length))
    model.add(Dropout(0.2))
    model.add(Conv1D(50, 3, padding='same', activation=mish, strides=2))
    model.add(GlobalAveragePooling1D())
    model.add(Flatten())
    model.add(Dropout(0.2))
    model.add(Dense(50, activation=mish))
    model.add(Dropout(0.2))
    model.add(Dense(n_class, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(learning_rate=0.002))

    return model

## 2. Hold Out

In [28]:
from sklearn.model_selection import train_test_split

In [29]:
X_train_holdout, X_test_holdout, y_train_holdout, y_test_holdout = train_test_split(x_train, y, test_size=0.2, random_state=123)

In [35]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=2, restore_best_weights=True)

In [ ]:
model = get_model()
history = model.fit(X_train_holdout, y_train_holdout, 
                    epochs           = 20,
                    callbacks        = [es],
                    batch_size       = 16,
                    validation_data = (X_test_holdout, y_test_holdout))

Epoch 1/20
2744/2744 [==============================] - 104s 38ms/step - loss: 1.1309 - val_loss: 0.8518
Epoch 2/20
2744/2744 [==============================] - 102s 37ms/step - loss: 0.7327 - val_loss: 0.7030
Epoch 3/20
2744/2744 [==============================] - 100s 37ms/step - loss: 0.5770 - val_loss: 0.6766
Epoch 4/20
2744/2744 [==============================] - 109s 40ms/step - loss: 0.4875 - val_loss: 0.6727
Epoch 5/20


## 3. K-Fold

In [ ]:

# 5-Fold 시행
from sklearn.model_selection import StratifiedKFold
cv = StratifiedKFold(n_splits=5, random_state=123, shuffle=True)

In [ ]:
validation_pred = np.zeros((y.shape[0], n_class))
test_pred = np.zeros((test.shape[0], n_class))

i = 0
for train_idx, val_idx in tqdm_notebook(cv.split(x_train, y)):
    print("{}-Fold" .format(i+1))
    X_train = x_train[train_idx]
    y_train = y[train_idx]

    X_validation = x_train[val_idx]
    y_validation = y[val_idx]

    CNN = get_model()

    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=2, restore_best_weights=True)

    CNN.fit(X_train, y_train,
             epochs           = 20,
             callbacks        = [es],
             batch_size       = 16,
             validation_data  = (X_validation, y_validation))
    
    validation_pred[val_idx, :] = CNN.predict(X_validation)
    test_pred += (CNN.predict(x_test) / 5)
    print('')

    i += 1

1-Fold
Epoch 1/20
2744/2744 [==============================] - 56s 21ms/step - loss: 1.1836 - val_loss: 0.8646
Epoch 2/20
2744/2744 [==============================] - 55s 20ms/step - loss: 0.8021 - val_loss: 0.7452
Epoch 3/20
2744/2744 [==============================] - 55s 20ms/step - loss: 0.6381 - val_loss: 0.6717
Epoch 4/20
2744/2744 [==============================] - 54s 20ms/step - loss: 0.5397 - val_loss: 0.6708
Epoch 5/20
2744/2744 [==============================] - 55s 20ms/step - loss: 0.4813 - val_loss: 0.6479
Epoch 6/20
2744/2744 [==============================] - 54s 20ms/step - loss: 0.4362 - val_loss: 0.6728
Epoch 7/20
2744/2744 [==============================] - 54s 20ms/step - loss: 0.4015 - val_loss: 0.6824
Epoch 00007: early stopping

2-Fold
Epoch 1/20
1426/2744 [==============>...............] - ETA: 25s - loss: 1.3277

In [ ]:
CNN.summary()

NameError: ignored

In [ ]:
from sklearn.metrics import log_loss, recall_score, confusion_matrix

In [ ]:
y_pred = np.argmax(validation_pred, axis=1)
y_real = y.reshape((1, -1))[0]

In [ ]:
confusion_matrix(y_pred, y_real)

array([[ 9945,   699,   877,   714,   879],
       [  510,  4969,   227,   311,   149],
       [ 1031,   783,  8567,  1160,   865],
       [ 1012,   578,  1187, 12482,   512],
       [  737,   193,   696,   396,  5400]])

In [ ]:
# log_loss

log_loss(pd.get_dummies(y_real), validation_pred)

0.6686359543276905

In [ ]:
# 결과 저장
sample_submission[['0','1','2','3','4']] = test_pred
sample_submission.to_csv('/content/drive/MyDrive/[데이콘] 소설 작가 분류 AI 경진대회/02_나의코드/01_딥러닝/Test예측결과/submission_14.csv', index = False, encoding = 'utf-8')